# Examining the overlap between antipsychotics analysis with main DE analysis

In [1]:
import pandas as pd

## Load DE genes results

In [4]:
ap = pd.read_csv("../../_m/genes/diffExpr_sz_APVctl_full.txt", sep='\t', index_col=0)
noap = pd.read_csv("../../_m/genes/diffExpr_sz_noAPVctl_full.txt", sep='\t', index_col=0)
degs = pd.read_csv("../../../_m/genes/diffExpr_szVctl_full.txt", sep='\t', index_col=0)

### FDR 5% comparison

In [8]:
def get_degs(df, fdr):
    return df[(df["adj.P.Val"] < fdr)]

In [10]:
len(set(get_degs(ap, 0.05).index) & set(get_degs(noap, 0.05).index))

330

In [13]:
len(set(get_degs(ap, 0.05).index) & set(get_degs(noap, 0.05).index)) / len(set(get_degs(noap, 0.05).index))

0.49624060150375937

In [14]:
len(set(get_degs(ap, 0.05).index) & set(get_degs(degs, 0.05).index))

1924

In [17]:
len(set(get_degs(ap, 0.05).index) & set(get_degs(degs, 0.05).index)) / len(set(get_degs(ap, 0.05).index))

0.7147102526002972

In [18]:
len(set(get_degs(noap, 0.05).index) & set(get_degs(degs, 0.05).index))

520

In [19]:
len(set(get_degs(noap, 0.05).index) & set(get_degs(degs, 0.05).index)) / len(set(get_degs(noap, 0.05).index))

0.7819548872180451